## Naive CUDA Kernel

In [ ]:
%%writefile matrix_gpu_naive.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

__global__ void matmul(float *a, float *b, float *c, int n) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (row < n && col < n) {
        float sum = 0.0f;
        for (int k = 0; k < n; k++) {
            sum += a[row * n + k] * b[k * n + col];
        }
        c[row * n + col] = sum;
    }
}

int main(int argc, char **argv) {
    int n = 512;
    if (argc > 1) {
        n = atoi(argv[1]);
    }

    printf("Matrix size: %d x %d\n", n, n);

    size_t size = n * n * sizeof(float);
    
    float *a = (float *)malloc(size);
    float *b = (float *)malloc(size);
    float *c = (float *)malloc(size);
    
    for (int i = 0; i < n * n; i++) {
        a[i] = rand() % 100 / 100.0f;
        b[i] = rand() % 100 / 100.0f;
    }
    
    float *d_a, *d_b, *d_c;
    
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);
    
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);
    
    dim3 threads(16, 16);
    dim3 blocks((n + 15) / 16, (n + 15) / 16);
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    cudaEventRecord(start);
    matmul<<<blocks, threads>>>(d_a, d_b, d_c, n);
    cudaEventRecord(stop);
    
    cudaEventSynchronize(stop);
    
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    
    printf("Time: %f ms\n", milliseconds);
    
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
    
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    
    free(a);
    free(b);
    free(c);

    return 0;
}

Overwriting matrix_gpu_naive.cu


## Running CUDA

In [2]:
!nvidia-smi

Sun Feb  1 05:58:36 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [4]:
!nvcc -arch=sm_75 matrix_gpu_naive.cu -o matrix_gpu_naive

In [5]:
!./matrix_gpu_naive 512
!./matrix_gpu_naive 1024
!./matrix_gpu_naive 2048
!./matrix_gpu_naive 4096
!./matrix_gpu_naive 8192

Matrix size: 512 x 512
Time: 1.222784 ms
Matrix size: 1024 x 1024
Time: 9.235648 ms
Matrix size: 2048 x 2048
Time: 74.927299 ms
Matrix size: 4096 x 4096
Time: 398.439148 ms
Matrix size: 8192 x 8192
Time: 2651.644531 ms


## Optimizing CUDA Code - Tiled CUDA kernel

In [6]:
%%writefile matrix_gpu_tiled.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#define TILE 16

__global__ void matmul_tiled(float *a, float *b, float *c, int n) {
    __shared__ float tile_a[TILE][TILE];
    __shared__ float tile_b[TILE][TILE];

    int tx = threadIdx.x;
    int ty = threadIdx.y;
    
    int row = blockIdx.y * TILE + ty;
    int col = blockIdx.x * TILE + tx;

    float sum = 0.0f;

    for (int m = 0; m < (n + TILE - 1) / TILE; m++) {
        if (row < n && (m * TILE + tx) < n)
            tile_a[ty][tx] = a[row * n + m * TILE + tx];
        else
            tile_a[ty][tx] = 0.0f;

        if (col < n && (m * TILE + ty) < n)
            tile_b[ty][tx] = b[(m * TILE + ty) * n + col];
        else
            tile_b[ty][tx] = 0.0f;

        __syncthreads();

        for (int k = 0; k < TILE; k++)
            sum += tile_a[ty][k] * tile_b[k][tx];

        __syncthreads();
    }

    if (row < n && col < n)
        c[row * n + col] = sum;
}

int main(int argc, char **argv) {
    int n = 512;
    if (argc > 1) {
        n = atoi(argv[1]);
    }

    printf("Matrix size: %d x %d\n", n, n);

    size_t size = n * n * sizeof(float);
    
    float *a = (float *)malloc(size);
    float *b = (float *)malloc(size);
    float *c = (float *)malloc(size);
    
    for (int i = 0; i < n * n; i++) {
        a[i] = rand() % 100 / 100.0f;
        b[i] = rand() % 100 / 100.0f;
    }
    
    float *d_a, *d_b, *d_c;
    
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);
    
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);
    
    dim3 threads(16, 16);
    dim3 blocks((n + 15) / 16, (n + 15) / 16);
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    cudaEventRecord(start);
    matmul_tiled<<<blocks, threads>>>(d_a, d_b, d_c, n);
    cudaEventRecord(stop);
    
    cudaEventSynchronize(stop);
    
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    
    printf("Time: %f ms\n", milliseconds);
    
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
    
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    
    free(a);
    free(b);
    free(c);

    return 0;
}

Overwriting matrix_gpu_tiled.cu


In [7]:
!nvcc -arch=sm_75 matrix_gpu_tiled.cu -o matrix_gpu_tiled

In [8]:
!./matrix_gpu_tiled 512
!./matrix_gpu_tiled 1024
!./matrix_gpu_tiled 2048
!./matrix_gpu_tiled 4096
!./matrix_gpu_tiled 8192


Matrix size: 512 x 512
Time: 0.322944 ms
Matrix size: 1024 x 1024
Time: 2.198464 ms
Matrix size: 2048 x 2048
Time: 46.329983 ms
Matrix size: 4096 x 4096
Time: 292.058533 ms
Matrix size: 8192 x 8192
Time: 1739.154175 ms


## Using cuBLAS Library



In [9]:
%%writefile matrix_gpu_cublas.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cublas_v2.h>

int main(int argc, char **argv) {
    int N;
    if (argc > 1) {
        N = atoi(argv[1]);
    } else {
        N = 1024; // default size
    }
    
    size_t size = N * N * sizeof(float);

    float *h_A = (float *)malloc(size);
    float *h_B = (float *)malloc(size);
    float *h_C = (float *)malloc(size);

    for (int i = 0; i < N * N; i++) {
        h_A[i] = rand() % 100 / 100.0f;
    }
    
    for (int i = 0; i < N * N; i++) {
        h_B[i] = rand() % 100 / 100.0f;
    }

    float *d_A;
    float *d_B;
    float *d_C;
    
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    cublasHandle_t handle;
    cublasCreate(&handle);

    float alpha = 1.0f;
    float beta  = 0.0f;

    cudaEvent_t start;
    cudaEvent_t stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);
    cublasSgemm(handle,
                CUBLAS_OP_N, CUBLAS_OP_N,
                N, N, N,
                &alpha,
                d_B, N,
                d_A, N,
                &beta,
                d_C, N);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    printf("cuBLAS SGEMM time (N=%d): %f ms\n", N, ms);

    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);   
    volatile float sink = h_C[0];

    cublasDestroy(handle);
    
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    free(h_A);
    free(h_B);
    free(h_C);

    return 0;
}

Overwriting matrix_gpu_cublas.cu


In [10]:
!nvcc matrix_gpu_cublas.cu -o matrix_gpu_cublas -lcublas

In [11]:
!./matrix_gpu_cublas 512
!./matrix_gpu_cublas 1024
!./matrix_gpu_cublas 2048
!./matrix_gpu_cublas 4096
!./matrix_gpu_cublas 8192

cuBLAS SGEMM time (N=512): 5.489536 ms
cuBLAS SGEMM time (N=1024): 6.142016 ms
cuBLAS SGEMM time (N=2048): 11.465792 ms
cuBLAS SGEMM time (N=4096): 53.734783 ms
cuBLAS SGEMM time (N=8192): 293.303375 ms


## Creating a Shared Library and Using it in Python

In [12]:
%%writefile matrix_lib.cu
#include <cuda_runtime.h>
#include <stdio.h>
#define TILE_WIDTH 16

__global__ void matrixMultiplyTiled(float *A, float *B, float *C, int N) {
    __shared__ float ds_A[TILE_WIDTH][TILE_WIDTH];
    __shared__ float ds_B[TILE_WIDTH][TILE_WIDTH];

    int tx = threadIdx.x;
    int ty = threadIdx.y;
    
    int Row = blockIdx.y * TILE_WIDTH + ty;
    int Col = blockIdx.x * TILE_WIDTH + tx;

    float Pvalue = 0.0f;

    int numTiles = (N + TILE_WIDTH - 1) / TILE_WIDTH;
    for (int m = 0; m < numTiles; ++m) {
        int aCol = m * TILE_WIDTH + tx;
        if (Row < N && aCol < N) {
            ds_A[ty][tx] = A[Row * N + aCol];
        } else {
            ds_A[ty][tx] = 0.0f;
        }

        int bRow = m * TILE_WIDTH + ty;
        if (Col < N && bRow < N) {
            ds_B[ty][tx] = B[bRow * N + Col];
        } else {
            ds_B[ty][tx] = 0.0f;
        }

        __syncthreads();

        for (int k = 0; k < TILE_WIDTH; ++k) {
            Pvalue += ds_A[ty][k] * ds_B[k][tx];
        }

        __syncthreads();
    }

    if (Row < N && Col < N) {
        C[Row * N + Col] = Pvalue;
    }
}

extern "C" void gpu_matrix_multiply(float *h_A, float *h_B, float *h_C, int N) {
    size_t size = N * N * sizeof(float);

    float *d_A;
    float *d_B;
    float *d_C;
    
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    dim3 block(TILE_WIDTH, TILE_WIDTH);
    
    int numBlocksX = (N + TILE_WIDTH - 1) / TILE_WIDTH;
    int numBlocksY = (N + TILE_WIDTH - 1) / TILE_WIDTH;
    dim3 grid(numBlocksX, numBlocksY);

    matrixMultiplyTiled<<<grid, block>>>(d_A, d_B, d_C, N);
    cudaDeviceSynchronize();
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
}

Writing matrix_lib.cu


In [13]:
!nvcc -Xcompiler -fPIC -shared matrix_lib.cu -o libmatrix.so

In [14]:
import ctypes
import numpy as np
import time
import sys
import os

In [23]:
lib = ctypes.cdll.LoadLibrary("./libmatrix.so")
lib.gpu_matrix_multiply.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags="C_CONTIGUOUS"),
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags="C_CONTIGUOUS"),
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags="C_CONTIGUOUS"),
    ctypes.c_int
]

N = 512

for i in range(5):
    print(f"\nTesting with matrix size N = {N}")
    
    A = np.random.rand(N, N).astype(np.float32)
    B = np.random.rand(N, N).astype(np.float32)
    C = np.zeros((N, N), dtype=np.float32)
    
    start = time.time()
    lib.gpu_matrix_multiply(A.ravel(), B.ravel(), C.ravel(), N)
    end = time.time()
    elapsed_time = end - start
    
    print(f"CUDA library time (N={N}): {elapsed_time * 1000:.4f} milliseconds")
    
    N = N * 2


Testing with matrix size N = 512
CUDA library time (N=512): 1.9896 milliseconds

Testing with matrix size N = 1024
CUDA library time (N=1024): 3.7229 milliseconds

Testing with matrix size N = 2048
CUDA library time (N=2048): 20.6728 milliseconds

Testing with matrix size N = 4096
CUDA library time (N=4096): 75.4676 milliseconds

Testing with matrix size N = 8192
CUDA library time (N=8192): 290.0529 milliseconds


## Adding Custom Functions to the Shared Library

In [16]:
%%writefile conv_gpu.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

__global__ void convolution2D_GPU(
    unsigned char *image,
    int *kernel,
    int *output,
    int width,
    int height
) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x >= width || y >= height) return;

    int sum = 0;
    int k = 1;  // 3x3 kernel radius

    for (int ky = -k; ky <= k; ky++) {
        for (int kx = -k; kx <= k; kx++) {
            int ix = x + kx;
            int iy = y + ky;
            if (ix >= 0 && ix < width && iy >= 0 && iy < height) {
                int pixel = image[iy * width + ix];
                int weight = kernel[(ky + k) * 3 + (kx + k)];
                sum += pixel * weight;
            }
        }
    }
    output[y * width + x] = sum;
}

int main() {
    int width = 512;
    int height = 512;
    size_t img_size = width * height * sizeof(unsigned char);
    size_t out_size = width * height * sizeof(int);

    unsigned char *h_img = (unsigned char*)malloc(img_size);
    int *h_out = (int*)malloc(out_size);

    int h_kernel[9] = {
        -1, -1, -1,
        -1,  8, -1,
        -1, -1, -1
    };

    for (int i = 0; i < width * height; i++)
        h_img[i] = rand() % 256;

    unsigned char *d_img;
    int *d_kernel, *d_out;

    cudaMalloc(&d_img, img_size);
    cudaMalloc(&d_kernel, 9 * sizeof(int));
    cudaMalloc(&d_out, out_size);

    cudaMemcpy(d_img, h_img, img_size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_kernel, h_kernel, 9 * sizeof(int), cudaMemcpyHostToDevice);

    dim3 block(16, 16);
    dim3 grid((width + 15) / 16, (height + 15) / 16);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);
    convolution2D_GPU<<<grid, block>>>(d_img, d_kernel, d_out, width, height);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    printf("CUDA convolution time: %f ms\n", ms);

    cudaMemcpy(h_out, d_out, out_size, cudaMemcpyDeviceToHost);

    cudaFree(d_img);
    cudaFree(d_kernel);
    cudaFree(d_out);
    free(h_img);
    free(h_out);

    return 0;
}


Writing conv_gpu.cu


In [17]:
!nvcc conv_gpu.cu -o conv_gpu

In [18]:
!./conv_gpu

CUDA convolution time: 7.186432 ms


#### Adding and Using the Convolution to your Custom Python Library

In [19]:
%%writefile conv_lib.cu
#include <cuda_runtime.h>
#include <stdio.h>

__global__ void convolution2D_GPU(
    unsigned char *image,
    int *kernel,
    int *output,
    int width,
    int height
) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x >= width || y >= height) return;

    int sum = 0;
    int k = 1; // 3x3 kernel radius

    for (int ky = -k; ky <= k; ky++) {
        for (int kx = -k; kx <= k; kx++) {
            int ix = x + kx;
            int iy = y + ky;

            if (ix >= 0 && ix < width && iy >= 0 && iy < height) {
                int pixel = image[iy * width + ix];
                int weight = kernel[(ky + k) * 3 + (kx + k)];
                sum += pixel * weight;
            }
        }
    }

    output[y * width + x] = sum;
}

extern "C" void gpu_convolution(
    unsigned char *h_image,
    int *h_kernel,
    int *h_output,
    int width,
    int height
) {
    size_t img_size = width * height * sizeof(unsigned char);
    size_t out_size = width * height * sizeof(int);

    unsigned char *d_image;
    int *d_kernel;
    int *d_output;

    cudaMalloc((void**)&d_image, img_size);
    cudaMalloc((void**)&d_kernel, 9 * sizeof(int));
    cudaMalloc((void**)&d_output, out_size);

    cudaMemcpy(d_image, h_image, img_size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_kernel, h_kernel, 9 * sizeof(int), cudaMemcpyHostToDevice);

    dim3 block(16, 16);
    dim3 grid((width + 15) / 16, (height + 15) / 16);

    convolution2D_GPU<<<grid, block>>>(d_image, d_kernel, d_output, width, height);
    cudaDeviceSynchronize();

    cudaMemcpy(h_output, d_output, out_size, cudaMemcpyDeviceToHost);

    cudaFree(d_image);
    cudaFree(d_kernel);
    cudaFree(d_output);
}


Writing conv_lib.cu


In [20]:
!nvcc -Xcompiler -fPIC -shared matrix_lib.cu conv_lib.cu -o libmatrix.so

In [21]:
!nm -D libmatrix.so | grep gpu_convolution

000000000000b0e5 T gpu_convolution


In [24]:
if './libmatrix.so' in sys.modules:
    del sys.modules['./libmatrix.so']

lib_path = os.path.abspath('./libmatrix.so')
lib = ctypes.CDLL(lib_path)

gpu_conv_func = lib.gpu_convolution
gpu_conv_func.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.uint8, ndim=1, flags="C_CONTIGUOUS"),
    np.ctypeslib.ndpointer(dtype=np.int32, ndim=1, flags="C_CONTIGUOUS"),
    np.ctypeslib.ndpointer(dtype=np.int32, ndim=1, flags="C_CONTIGUOUS"),
    ctypes.c_int,
    ctypes.c_int
]

width = 512
height = 512

for i in range(5):
    print(f"\nTesting with image size: {width} x {height}")
    
    image = np.random.randint(0, 256, size=(height, width), dtype=np.uint8)
    
    kernel = np.array([
        -1, -1, -1,
        -1,  8, -1,
        -1, -1, -1
    ], dtype=np.int32)
    
    output = np.zeros((height, width), dtype=np.int32)
    
    start = time.time()
    gpu_conv_func(
        image.ravel(),
        kernel,
        output.ravel(),
        width,
        height
    )
    end = time.time()
    elapsed_time = end - start
    
    print(f"CUDA convolution time ({width}x{height}): {elapsed_time * 1000:.4f} milliseconds")
    
    width = width * 2
    height = height * 2


Testing with image size: 512 x 512
CUDA convolution time (512x512): 1.4246 milliseconds

Testing with image size: 1024 x 1024
CUDA convolution time (1024x1024): 3.8025 milliseconds

Testing with image size: 2048 x 2048
CUDA convolution time (2048x2048): 13.1280 milliseconds

Testing with image size: 4096 x 4096
CUDA convolution time (4096x4096): 51.6033 milliseconds

Testing with image size: 8192 x 8192
CUDA convolution time (8192x8192): 190.6698 milliseconds
